In [54]:
import pandas as pd
from datetime import datetime, date
%matplotlib inline

In [70]:
query = ('https://data.cityofchicago.org/resource/ct72-a55e.json?$limit=50000')
predictions = pd.read_json(query)
cols_to_delete = [_ for _ in predictions.columns if _.startswith(':@')]
print('Deleting columns: %s' % ', '.join(cols_to_delete))
predictions = predictions.drop(cols_to_delete, axis=1)
predictions.head()

Deleting columns: 


,beach_name,date,latitude,location,longitude,predicted_level,prediction_source,recordid
0,12th Street,2017-05-26,41.8638,"{'type': 'Point', 'coordinates': [-87.6082, 41...",-87.6082,160.0,DNA Model,12thStreet20170526
1,12th Street,2017-05-27,41.8638,"{'type': 'Point', 'coordinates': [-87.6082, 41...",-87.6082,54.7,DNA Model,12thStreet20170527
2,12th Street,2017-05-28,41.8638,"{'type': 'Point', 'coordinates': [-87.6082, 41...",-87.6082,85.9,DNA Model,12thStreet20170528
3,12th Street,2017-05-29,41.8638,"{'type': 'Point', 'coordinates': [-87.6082, 41...",-87.6082,109.5,DNA Model,12thStreet20170529
4,12th Street,2017-05-30,41.8638,"{'type': 'Point', 'coordinates': [-87.6082, 41...",-87.6082,53.2,DNA Model,12thStreet20170530


In [56]:
# Dropping more columns I don't care about.  Note, I'll get location from another table in a sec...
predictions = predictions.drop(['recordid', 'prediction_source', 'latitude', 'longitude', 'location'], axis=1)

In [57]:
# These are the datatypes I have
predictions.dtypes

beach_name                 object
date               datetime64[ns]
predicted_level           float64
dtype: object

In [58]:
if len(predictions) >= 1000:
    print("WARNING: Maxed out predictions from API - need pagination")

In [59]:
predictions['date'] = predictions['date'].dt.date
predictions.head()

,beach_name,date,predicted_level
0,12th Street,2017-05-26,160.0
1,12th Street,2017-05-27,54.7
2,12th Street,2017-05-28,85.9
3,12th Street,2017-05-29,109.5
4,12th Street,2017-05-30,53.2


In [71]:
from urllib.parse import quote
query = "https://data.cityofchicago.org/resource/awhh-mb2r.json?$where=dna_sample_timestamp%20%3E%20%222017-01-01T00:00:00%22&$order=dna_sample_timestamp%20desc&$limit=50000"
lab_results = pd.read_json(query)
lab_results.head()

,beach,dna_reading_mean,dna_sample_1_reading,dna_sample_2_reading,dna_sample_timestamp,dna_test_id,latitude,location,longitude
0,Calumet,162.628411,152,174.0,2017-06-23T00:00:00.000,3083,41.7142,"{'type': 'Point', 'coordinates': [-87.5299, 41...",-87.5299
1,Osterman,538.273165,457,634.0,2017-06-23T00:00:00.000,3192,41.9877,"{'type': 'Point', 'coordinates': [-87.6545, 41...",-87.6545
2,Montrose,1386.880673,1158,1661.0,2017-06-23T00:00:00.000,3190,41.9655,"{'type': 'Point', 'coordinates': [-87.6385, 41...",-87.6385
3,63rd Street,5393.683435,21923,1327.0,2017-06-23T00:00:00.000,3205,41.7827,"{'type': 'Point', 'coordinates': [-87.5748, 41...",-87.5748
4,Foster,203.749847,222,187.0,2017-06-23T00:00:00.000,3191,41.9785,"{'type': 'Point', 'coordinates': [-87.6515, 41...",-87.6515


In [61]:
#Since this is only 2017, there are only DNA data
lab_results = lab_results[['beach', 'dna_reading_mean', 'dna_sample_timestamp', 'dna_test_id', 'location']]
lab_results['dna_sample_timestamp'] = pd.to_datetime(lab_results['dna_sample_timestamp'])
lab_results.head()

,beach,dna_reading_mean,dna_sample_timestamp,dna_test_id,location
0,Calumet,162.628411,2017-06-23,3083,"{'type': 'Point', 'coordinates': [-87.5299, 41..."
1,Osterman,538.273165,2017-06-23,3192,"{'type': 'Point', 'coordinates': [-87.6545, 41..."
2,Montrose,1386.880673,2017-06-23,3190,"{'type': 'Point', 'coordinates': [-87.6385, 41..."
3,63rd Street,5393.683435,2017-06-23,3205,"{'type': 'Point', 'coordinates': [-87.5748, 41..."
4,Foster,203.749847,2017-06-23,3191,"{'type': 'Point', 'coordinates': [-87.6515, 41..."


In [62]:
# These are the beaches that are not being predicted.  Humboldt because it is not on the lake, and the
# rest because they 
beaches_wo_predictions = list(set(lab_results['beach'].unique()) - set(predictions['beach_name'].unique()))
beaches_wo_predictions

['South Shore', 'Montrose', 'Rainbow', 'Calumet', 'Humboldt', '63rd Street']

In [63]:
# Juneway beach is being predicted, but not measured.  Weird, but it is small and nestled among others,
# so let's ignore it for now
set(predictions['beach_name'].unique()) - set(lab_results['beach'].unique())

{'Juneway'}

In [64]:
lab_results.dtypes

beach                           object
dna_reading_mean               float64
dna_sample_timestamp    datetime64[ns]
dna_test_id                      int64
location                        object
dtype: object

In [65]:
lab_results['date'] = lab_results['dna_sample_timestamp'].dt.date
lab_results['beach_name'] = lab_results['beach']
del lab_results['beach']
lab_results.head()

,dna_reading_mean,dna_sample_timestamp,dna_test_id,location,date,beach_name
0,162.628411,2017-06-23,3083,"{'type': 'Point', 'coordinates': [-87.5299, 41...",2017-06-23,Calumet
1,538.273165,2017-06-23,3192,"{'type': 'Point', 'coordinates': [-87.6545, 41...",2017-06-23,Osterman
2,1386.880673,2017-06-23,3190,"{'type': 'Point', 'coordinates': [-87.6385, 41...",2017-06-23,Montrose
3,5393.683435,2017-06-23,3205,"{'type': 'Point', 'coordinates': [-87.5748, 41...",2017-06-23,63rd Street
4,203.749847,2017-06-23,3191,"{'type': 'Point', 'coordinates': [-87.6515, 41...",2017-06-23,Foster


In [66]:
#Left joining on lab_results because we want all beaches, even the ones that aren't being predicted.
beach_days = lab_results.merge(predictions, how='left', on=['beach_name','date'])
beach_days.head()

,dna_reading_mean,dna_sample_timestamp,dna_test_id,location,date,beach_name,predicted_level
0,162.628411,2017-06-23,3083,"{'type': 'Point', 'coordinates': [-87.5299, 41...",2017-06-23,Calumet,NaN
1,538.273165,2017-06-23,3192,"{'type': 'Point', 'coordinates': [-87.6545, 41...",2017-06-23,Osterman,290.3
2,1386.880673,2017-06-23,3190,"{'type': 'Point', 'coordinates': [-87.6385, 41...",2017-06-23,Montrose,NaN
3,5393.683435,2017-06-23,3205,"{'type': 'Point', 'coordinates': [-87.5748, 41...",2017-06-23,63rd Street,NaN
4,203.749847,2017-06-23,3191,"{'type': 'Point', 'coordinates': [-87.6515, 41...",2017-06-23,Foster,275.0


In [67]:
# Verify that only the six beaches with no predictions still have no predictions
beach_days.loc[beach_days['predicted_level'].isnull()]['beach_name'].unique()

array(['Calumet', 'Montrose', '63rd Street', 'Rainbow', 'Humboldt',
       'South Shore'], dtype=object)

In [68]:
del beach_days['dna_sample_timestamp']
beach_days.head()

,dna_reading_mean,dna_test_id,location,date,beach_name,predicted_level
0,162.628411,3083,"{'type': 'Point', 'coordinates': [-87.5299, 41...",2017-06-23,Calumet,NaN
1,538.273165,3192,"{'type': 'Point', 'coordinates': [-87.6545, 41...",2017-06-23,Osterman,290.3
2,1386.880673,3190,"{'type': 'Point', 'coordinates': [-87.6385, 41...",2017-06-23,Montrose,NaN
3,5393.683435,3205,"{'type': 'Point', 'coordinates': [-87.5748, 41...",2017-06-23,63rd Street,NaN
4,203.749847,3191,"{'type': 'Point', 'coordinates': [-87.6515, 41...",2017-06-23,Foster,275.0


In [69]:
print("There are %s beaches" % str(len(beach_days['beach_name'].unique())))

There are 20 beaches
